# get dataframe stats

In [ ]:
stats = final_df.describe(include='all', percentiles=[0.05, 0.25, 0.5, 0.75, 0.99])

stats.loc['dtypes'] = final_df.dtypes
stats.loc['missing_rate'] = final_df.isnull().mean()

stats.T.reset_index()\
        .rename(columns={'index': 'column_name'})\
        .sort_values(by='column_name')\
        .to_csv('edge_simu_driver_stats.csv', index=False)

# set cell value conditionally

In [ ]:
# config pd dataframe
config['rm_reason'] = None


# high missing rate
config.loc[
    config[(config['columnFlag'] == 'Candidate') 
           &(config['missingPercentage'] > 0.99)].index,
    'rm_reason',
] = 'high missing rate'

# low iv
config.loc[
    config[(config['columnFlag'] == 'Candidate') 
           &(config['iv'] < 0.01)].index,
    'rm_reason',
] = 'low iv'


# get model score difference

In [ ]:
import pandas as pd
import numpy as np


# get diff stats
bins = [-np.inf, -100, -50, -10, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 10, 50, 100, np.inf]

diff_stats_df = pd.DataFrame([], columns=[
    'diff_category',
    'mean',
    'min',
    'max',
    'std',
    'total_cnt',
    'null_cnt',
]
+ ['p0.1', 'p1'] + [f"p{i}" for i in range(0, 100, 5)[1:]] + ['p99']
+ [f'bin_boundary_{b}' for b in range(len(bins)-1)] + [f'bin_hist_{b}' for b in range(len(bins)-1)]
)


def diff_stats(diff, diff_category):
    diff = pd.DataFrame(diff.rename('diff'))
    row = {}
    row['diff_category'] = diff_category
    row['mean'] = diff['diff'].mean()
    row['min'] = diff['diff'].min()
    row['max'] = diff['diff'].max()
    row['std'] = diff['diff'].std()
    row['total_cnt'] = len(diff)
    row['null_cnt'] = len(diff[diff['diff'].isna()])

    for c in diff_stats_df.columns:
        if 'p' not in c:
            continue
        p = float(c.lstrip('p')) / 100
        row[c] = diff['diff'].quantile(p)
        
    # hist
    bin_hist, bin_boundary = np.histogram(diff['diff'], bins=bins)
    bin_hist = bin_hist / sum(bin_hist)
    
    for i, a in enumerate(bin_hist):
        if np.isinf(bin_boundary[i]):
            row[f'bin_boundary_{i}'] = '_INF'
        else:
            row[f'bin_boundary_{i}'] = str(bin_boundary[i])
        row[f'bin_hist_{i}'] = a
    
    return row


df= None # dataframe contains model scores
# overall
diff = df['bt23_model_score_new_n'] - df['bt23_model_score_origin_n']
row = diff_stats(diff, 'overall_new_origin')

# read excel

In [ ]:
simu_excel_data_path = ''

df = pd.read_excel(simu_excel_data_path,
                   sheet_name='Sheet1',
                   engine='openpyxl',
                  )


# append row to dataframe

In [ ]:
row = {
    'key': val,
}

df = df.append(row, ignore_index = True)